In [ ]:
%pip install time
%pip install pandas
%pip install BeautifulSoup
%pip install openpyxl
%pip install requests
%pip install openpyxl
%pip install bs4
%pip install pyOpenSSL

In [1]:
import time
import pandas as pd
from datetime import datetime,timedelta
import re

from bs4 import BeautifulSoup
import requests

### 변수 (공무원+키워드)

In [16]:
#키워드
k = pd.read_csv('키워드.txt')
keywords = k['키워드'].values.tolist()

#지자체
f = pd.read_csv('경기.txt')
regions = f['경기'].values.tolist()

#업무명
w = pd.read_csv('업무명.txt')
works = w['업무'].values.tolist()

#사이트1
site1 = pd.read_csv('경기변수1.txt', sep='\t')
site1 = site1.fillna('')

#사이트2
site2 = pd.read_csv('경기변수2.txt', sep='\t')
site2 = site2.fillna('')

### time 복잡ver

In [20]:
names = []
article_urls = []
titles = []
days = []
content = []

for j in range(len(site1)):
    url = site1['url'][j]  #1
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    result = soup.find_all(site1['result1'][j], attrs = {site1['result2'][j] : site1['result3'][j]}) #2 #3 #4
    if len(result) == 0:
        article_urls.append('')
        titles.append('')
        days.append('')
        content.append('')
        names.append('')
    else:
        for i in range(len(result) - 1):
            name = site1['신문사'][j]
            names.append(name)
            idx=result[i].find('a')['href'].find('/')
            if idx == -1: idx = 0
            article_url = site1['article_url'][j] + result[i].find('a')['href'][idx:] #5
            article_urls.append(article_url)
            response=requests.get(article_url)
            response.encoding = site1['encoding'][j]  #6
            soup = BeautifulSoup(response.text, 'html.parser')
            title = soup.find(site1['title1'][j], attrs = {site1['title2'][j] : site1['title3'][j]}).text  #7 #8 #9
            titles.append(title)
            time = soup.find(site1['time1'][j],attrs = {site1['time2'][j] : site1['time3'][j]}).find_all(site1['time4'][j])[site1['time5'][j]].text #10 #11 #12 #13 #14
            #time = time[3:]
            days.append(time)
            article = soup.find_all(site1['article1'][j], attrs = {site1['article2'][j]:site1['article3'][j]}) #15
            body = ''
            for p in article:
                body += p.text
                body  += ' '
            content.append(body)

### time 간단ver

In [21]:
for j in range(len(site2)):
    url = site2['url'][j]  #1
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    result = soup.find_all(site2['result1'][j], attrs = {site2['result2'][j] : site2['result3'][j]}) #2 #3 #4
    if len(result) == 0:
        article_urls.append('')
        titles.append('')
        days.append('')
        content.append('')
        names.append('')
    else:
        for i in range(len(result)-1):
            name = site2['신문사'][j]
            names.append(name)
            idx=result[i].find('a')['href'].find('/')
            if idx == -1: idx = 0
            article_url = site2['article_url'][j] + result[i].find('a')['href'][idx:] #5
            article_urls.append(article_url)
            response=requests.get(article_url)
            response.encoding = site2['encoding'][j]  #6
            soup = BeautifulSoup(response.text, 'html.parser')
            title = soup.find(site2['title1'][j],attrs = {site2['title2'][j] : site2['title3'][j]}).text  #7 #8 #9
            titles.append(title)
            time = soup.find(site2['time1'][j],attrs = {site2['time2'][j] : site2['time3'][j]}).text #10 #11 #12
            #time = time[3:]
            days.append(time)
            article = soup.find_all(site2['article1'][j], attrs = {site2['article2'][j]:site2['article3'][j]}) #13
            body = ''
            for p in article:
                body += p.text
                body  += ' '
            content.append(body)

### 최후 DF 만들기

In [61]:
기사_경기 = pd.DataFrame({'신문사':names, '제목': titles, '날짜': days, 'url': article_urls, '본문': content})

In [62]:
#### 날짜 형식 맞추기
for i in range(len(기사_경기)):
    info = re.sub(r'[^0-9]', '', 기사_경기['날짜'][i])
    기사_경기['날짜'][i] = info[0:8]

#### 어제, 오늘 날짜인 기사만
from datetime import datetime
기사_경기_어제 = 기사_경기[(datetime.today() - timedelta(1)).strftime("%Y%m%d") == 기사_경기['날짜']].reset_index(drop=True)
기사_경기_오늘 = 기사_경기[기사_경기['날짜'] == datetime.today().strftime("%Y%m%d")].reset_index(drop=True)
기사_경기1 = pd.concat([기사_경기_어제, 기사_경기_오늘]).reset_index(drop=True)
기사_경기2 = 기사_경기1

In [63]:
# 공무원 + 지역 + 기본 키워드

for i, txt in enumerate(기사_경기1['본문']):
    score = any(keyword in txt for keyword in keywords)+any(region in txt for region in regions)  #+any(work in txt for work in works)
    if score != 2:
        기사_경기1 = 기사_경기1.drop(index = i, axis = 0)

In [64]:
# 중복되는 내용 지우기
기사_경기1 = 기사_경기1.drop_duplicates(subset='본문', ignore_index = True)
기사_경기1 = 기사_경기1.sort_values(by = '날짜', ascending = False)

#기사_경기1['본문'] = [re.sub('[^A-Za-z0-9가-힣\s]', '', s) for s in 기사_경기1['본문']]
기사_경기1['본문'] = [re.sub("\n", "", s) for s in 기사_경기1['본문']]

# 제거
기사_경기1['본문'] = [s.replace(u'\xa0', u'') for s in 기사_경기1['본문']]
# 기사_경기1['본문'] = [s.replace(' ', '') for s in 기사_경기1['본문']]


기사_경기_지역기본 = 기사_경기1.iloc[:,0:4]
기사_경기_지역기본.to_excel('경기_지역+기본.xlsx', index = False)

In [65]:
# 공무원 + 지역 + 업무 키워드

for i, txt in enumerate(기사_경기2['본문']):
    score = any(region in txt for region in regions) + any(work in txt for work in works)
    if score != 2:
        기사_경기2 = 기사_경기2.drop(index = i, axis = 0)

In [67]:
# 중복되는 내용 지우기
기사_경기2 = 기사_경기2.drop_duplicates(subset='본문', ignore_index = True)
기사_경기2 = 기사_경기2.sort_values(by = '날짜', ascending = False)

기사_경기2['본문'] = [re.sub('[^A-Za-z0-9가-힣\s]', '', s) for s in 기사_경기2['본문']]
기사_경기2['본문'] = [re.sub("\n", "", s) for s in 기사_경기2['본문']]

# 제거
기사_경기2['본문'] = [s.replace(u'\xa0', u'') for s in 기사_경기2['본문']]
# 기사_경기2['본문'] = [s.replace(' ', '') for s in 기사_경기2['본문']]

        
기사_경기_지역업무 = 기사_경기2.iloc[:,0:4]
기사_경기_지역업무.to_excel('경기_지역+업무.xlsx', index = False)